In [ ]:
%matplotlib inline
import platform
print('the python version is: ' + platform.python_version())
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext

In [ ]:
print('\n' + ' //// Started local Spark session //// ' + '\n')
sc = SparkContext("local", "RDD basic functions App")
sc.setLogLevel("WARN")

Load data from World Wide Web and store it in *kddcup.data_10_percent.gz*:

Information about data schema can be found here: http://kdd.ics.uci.edu/databases/kddcup99/task.html

In [ ]:
import urllib.request
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "./kddcup.data_10_percent.gz"
#construct the RDD from input file
rawData = sc.textFile(data_file)

Show first 5 lines:

In [ ]:
firstFiveLines = rawData.take(5)
for line in firstFiveLines:
    print(line)

In [ ]:
#filter out normal connections - transformation
normalData = rawData.filter(lambda line: 'normal.' in line)

In [ ]:
#filter out normal connections - transformation + action
normalDataCount = normalData.count()
print(normalDataCount)

**Exercise:** Count the "normal" entries in connection data set and compare to total number 

In [ ]:
rawDataCount = rawData.count()
print('the total number of entries in data set is: {}'.format(rawDataCount) + ' compared to {} '.format(normalDataCount) + 'normal entries.')

**Exercise:** Measure the time needed to formulate transformation and combination to transformation (filter) plus action (counting). Hint: use the 'time' module

In [ ]:
#measure time to complete full map reduce
from time import time
t0 = time()
normalData = rawData.filter(lambda x: 'normal.' in x)
t1 = time()
t_trans = t1-t0
t0 = time()
normalDataCount = rawData.filter(lambda x: 'normal.' in x).collect()
t1 = time()
t_transAct = t1-t0
print('transformation time: {}'.format(round((t_trans)*1000,3)) + ' ms, full time: {}'.format(round((t_transAct)*1000,3)) +' ms')

To avoid overhead, sometimes it is sufficient to get an approximation for an amount. For this, a sub-sample or the data set might be sufficient to get approximate values. 

**Exercise:** Use the 'sample' transformation to collect a subset of the full data set and repeat time measurement. What is your observation? Vary the sample size starting with 10%.

In [ ]:
t0 = time()
rawDataSamples = rawData.sample(False, 0.1, 1234)
samplesCount = rawDataSamples.count()
t1 = time()
t_sample = t1-t0
print('Sample size is {}'.format(samplesCount) + ' of {}'.format(rawDataCount))
print('sample full time: {}'.format(round((t_sample)*1000,3)) + ' ms, full time: {}'.format(round((t_transAct)*1000,3)) +' ms')

Get the suspicious data sets by using the 'subtract' transformation.

In [ ]:
suspiciousData = rawData.subtract(normalData)
suspiciousDataCount = suspiciousData.count()
print('Number of suspicious entries  {}'.format(suspiciousDataCount))

**Exercise:** Collect all different types of connection protocols and used services in the data set (you can also use the sample data set instead of the full one). Hint: you have to first split up each individual entry, since it is simply a long string. Use the 'split()' function in a mapping transformation first.

In [ ]:
inData = rawData.map(lambda line: line.split(","))
print('take first element each')
print(rawData.take(1))
print(inData.take(1))

In [ ]:
protocols = inData.map(lambda x: x[1]).distinct()
print('all protcols found in the data set: {} '.format(protocols.collect()) )

In [ ]:
services = inData.map(lambda x: x[2]).distinct()
print( services.collect() )

In [ ]:
product = protocols.cartesian(services).collect()
print('There are {} combinations of protocol X service'.format(len(product)))

In [ ]:
combinations = inData.map(lambda x: (x[1],x[2])).distinct()
print('There are {} combinations of protocols and services'.format(combinations.count()))

Show all protocol/service combinations which do not exist in the data set:

In [ ]:
print(protocols.cartesian(services).subtract(combinations).collect())

In [ ]:
sc.stop()